In [1]:
import pandas as pd
import re
from datetime import date
from datetime import datetime
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

In [2]:
df = pd.read_csv("../../data/bac_news.csv")

# GENERAL

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   link       67 non-null     object
 1   title      67 non-null     object
 2   date       67 non-null     object
 3   paragraph  67 non-null     object
dtypes: object(4)
memory usage: 2.2+ KB


In [4]:
df.describe(include='object')

,link,title,date,paragraph
count,67,67,67,67
unique,67,66,11,67
top,https://www.baccredomatic.com/es-cr/acerca-de-...,Vecinos de Pacuarito de Pérez Zeledón están li...,"3 de December, 2024",Dos empresas sociales fueron inauguradas duran...
freq,1,2,25,1


Calculates the number of missing (null/NaN) values in each column of the DataFrame df. It returns a Series where the index is the column name and the value is the count of nulls in that column.

In [5]:
df.isnull().sum()

link         0
title        0
date         0
paragraph    0
dtype: int64

Counts the number of duplicate rows in the DataFrame df. It returns an integer representing how many rows are exact duplicates of previous rows. This helps you identify if your dataset has repeated entries.

In [6]:
df.duplicated().sum()

np.int64(0)

# LINKS

Check invalid links

In [7]:
df['link'].apply(lambda x: not x.startswith('http')).sum()

np.int64(0)

Check duplicated links

In [8]:
df['link'].duplicated().sum()

np.int64(0)

Remove duplicates

In [9]:
df = df.drop_duplicates(subset='link')

# DATE

Verify all dates have the following format: Month Day, Year (for example: Abr 2, 2024).

In [10]:
def verify_date_format(date):
    if pd.isna(date):  # Check for NaN/None values
        return True
    if not isinstance(date, str):  # Check if value is not a string
        return True
    return not bool(re.match(r'[a-zA-Z]+\s\d+\s.\s\d+', date))

df_bad_format = df[df['date'].apply(verify_date_format)]

df_bad_format["date"].apply(lambda x: print(x))

23 de December, 2025
22 de December, 2025
22 de December, 2025
22 de December, 2025
22 de December, 2025
22 de December, 2025
25 de April, 2025
25 de April, 2025
25 de April, 2025
25 de April, 2025
25 de April, 2025
2 de January, 2025
2 de January, 2025
2 de January, 2025
10 de December, 2024
10 de December, 2024
10 de December, 2024
16 de December, 2024
16 de December, 2024
16 de December, 2024
16 de December, 2024
16 de December, 2024
16 de December, 2024
3 de December, 2024
4 de December, 2024
21 de January, 2025
21 de January, 2025
21 de January, 2025
3 de December, 2024
3 de December, 2024
3 de December, 2024
3 de December, 2024
3 de December, 2024
4 de December, 2024
3 de December, 2024
21 de January, 2025
3 de December, 2024
3 de December, 2024
21 de January, 2025
21 de January, 2025
3 de December, 2024
3 de December, 2024
3 de December, 2024
3 de December, 2024
21 de January, 2025
21 de January, 2025
7 de November, 2024
21 de January, 2025
3 de December, 2024
3 de December, 202

0     None
1     None
2     None
3     None
4     None
      ... 
62    None
63    None
64    None
65    None
66    None
Name: date, Length: 67, dtype: object

Some dates where extracted with the text. The idea here is to extract only the date that is in the following format: Month Day, Year (for example: Abr 2, 2024) and change it to a date format.

In [11]:
# Clean dates
temp_dates = []

for date in df['date']:
    try:
        new_date = datetime.strptime(date.strip(), '%d de %B, %Y')
        temp_dates.append(new_date)
    except Exception as e:
        print(f"Error parsing date: {date} - {e}")
        temp_dates.append(None)

df['new_dates'] = temp_dates

Just to verify

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   link       67 non-null     object        
 1   title      67 non-null     object        
 2   date       67 non-null     object        
 3   paragraph  67 non-null     object        
 4   new_dates  67 non-null     datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 2.7+ KB


# TEXTS

In [13]:
paragraphs_na = df[df["paragraph"].isna()]["link"]
print(paragraphs_na)

Series([], Name: link, dtype: object)


In [14]:
df.describe()

,new_dates
count,67
mean,2025-01-11 05:00:53.731343360
min,2021-05-20 00:00:00
25%,2024-12-03 00:00:00
50%,2024-12-16 00:00:00
75%,2025-01-21 00:00:00
max,2025-12-23 00:00:00


In [15]:
df['title_len'] = df['title'].str.len()
df['paragraph_len'] = df['paragraph'].str.len()

df[['title_len', 'paragraph_len']].describe()

,title_len,paragraph_len
count,67.000000,67.000000
mean,71.492537,1569.671642
std,20.020726,400.153584
min,12.000000,738.000000
25%,59.500000,1339.500000
50%,72.000000,1551.000000
75%,83.000000,1705.000000
max,120.000000,3527.000000


Here I look for any posibilities to clean the paragraphs from these dates + cities combinations. I had to include different combinations using REGEX.

In [41]:
def clean_paragraph_format(text):
    match = re.search(r'^(San José|Puntarenas),(\s\w*\sdel\s\d*\.|\s\w*\s\d*\.)', text)
    if match:
        text = text[match.end():].strip()
        text = re.sub(r'(-+)|(—)', '', text)
    text = re.sub(r'^(Noviembre|Octubre|Junio|Febrero|Marzo|Setiembre|San José)(,|\sde|)(\s\d*\.\s)', '', text)
    text = re.sub(r'● ', '', text)    
    return text

In [42]:
new_paragraphs = []
for paragraph in df['paragraph']:
    new_paragraphs.append(clean_paragraph_format(paragraph))

df['new_paragraph'] = new_paragraphs

In [37]:
df['new_paragraph_len'] = df['new_paragraph'].str.len()
df[['new_paragraph_len']].describe()

,new_paragraph_len
count,67.000000
mean,1562.074627
std,398.289879
min,738.000000
25%,1339.500000
50%,1539.000000
75%,1700.500000
max,3512.000000


In [39]:
df['new_paragraph'] = df['new_paragraph'].fillna('')